In [53]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 

In [6]:
m = convert_mat_np("m")
f = convert_mat_np("f")

In [7]:
def convert_mat_np(gender=None):
    f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
    m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat"
    # m_file = "../raw_data/Emotrans1_Boy_data_preprocessed_33.mat"
    # f_file = "../raw_data/Emotrans1_girl_data_preprocessed_33.mat"
    if gender == "f":
        data_dict_female = mat73.loadmat(f_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])
    elif gender == 'm':
        data_dict_female = mat73.loadmat(m_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])  
    else: 
        raise Exception("gender not defined")

In [8]:
# boy 18, 19, 21
# delete and put to the end of firl 
new_f = [] 
for i in range(f.shape[0]):
    new_f.append(f[i])
for i in [17,18,20]:
    new_f.append(m[i])
f = np.array(new_f)
# new_arr = []
m = np.delete(m,[17,18,20],axis=0)
print(f.shape)
print(m.shape)
# deleted.shape

(22, 4, 4)
(20, 4, 4)


In [9]:
# type: 0 --> average 3+4 and 1+2; 1--> use only 3 and 2  
def get_pos_or_neg(data,position,type=0):
    arr = []
    index = 0
    for i in range(data.shape[0]):
#         print(data[i].shape)
        if type == 0:
            diff = (data[i][position[0][0]][position[0][1]]+data[i][position[1][0]][position[1][1]])/2 - (data[i][position[3][0]][position[3][1]]+data[i][position[2][0]][position[2][1]])/2
        elif type == 1:
             diff = (data[i][position[1][0]][position[1][1]] - data[i][position[2][0]][position[2][1]])/2
        arr.append(diff)
    return np.array(arr)

In [ ]:
# select frequenceis and (0-4s -> 0 or 0.5-4.5s -> 1)
# output: x 
def choose_freq(data=None,freq = None, sec=None):
    if freq:
        data = np.delete(data,freq,axis=3)
    if sec == 0:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 1, axis=3)
        return data
    elif sec == 1:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 0, axis=3) 
        return data
    else:
        print('sec not specified')
        return data 

In [19]:
# reshape to 128*8*2 = 2048
# return x 
def squeeze_feature_size(data):
    new_data = []
    for d in range(data.shape[0]):
#         for i in range(df[d].shape[0]):
        size = data[d].shape
        new_data.append(data[d].reshape(size[0], size[1]*size[2]*size[3]))  
    return np.array(new_data)

In [23]:
# generate labels for x 
# input: data and label_type (0:zeros, 1: ones )
def generate_labels(df,type=None):
    arr = []
    label = []
    label_type = -1
    if type == 0:
        label_type = 0
    elif type == 1:
        label_type = 1
    else:
        print("Wrong type - get_pos_or_neg()")
        return 
    for d in range(df.shape[0]):
#         for i in range(df[d].shape[0]):
        size = df[d].shape
#         print(size)
        if label_type == 1:
            label.append(np.ones((size[0],1)))
        else:
            label.append(np.zeros((size[0],1)))
    return np.array(label)

In [28]:
# combine neg and postive for boys and girls separately
def combine_net_neg(netX,netY,negX,negY):
    new_x, new_y = [],[]
    for i in range(netX.shape[0]):
        new_x.append(np.concatenate((netX[i],negX[i]),axis=0))
        new_y.append(np.concatenate((netY[i],negY[i]),axis=0))
    return np.array(new_x),np.array(new_y)

In [33]:
def get_x_y_train(x,y):
    new_x = x[0]
    new_y = y[0]
    for i in range(1,len(x)):
        # print(new_x.shape)
        new_x=np.append(new_x,x[i],axis=0)
        new_y=np.append(new_y,y[i],axis=0)
    return new_x,new_y

In [48]:
# define model 
model = RandomForestClassifier(n_estimators=150) 

In [16]:
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
pos_df_m = get_pos_or_neg(m,pos,1)
neg_df_m = get_pos_or_neg(m,neg,1)
pos_df_f = get_pos_or_neg(f,pos,1)
neg_df_f = get_pos_or_neg(f,neg,1)


In [20]:
pos_sequeezed_m = squeeze_feature_size(pos_df_m)
neg_sequeezed_m = squeeze_feature_size(neg_df_m)
pos_sequeezed_f = squeeze_feature_size(pos_df_f)
neg_sequeezed_f = squeeze_feature_size(neg_df_f)

In [24]:
pos_labels_m = generate_labels(pos_sequeezed_m,1)
neg_labels_m = generate_labels(neg_sequeezed_m,0)
pos_labels_f = generate_labels(pos_sequeezed_f,1)
neg_labels_f = generate_labels(neg_sequeezed_f,0)

In [29]:
boy_x,boy_y = combine_net_neg(pos_sequeezed_m,pos_labels_m,neg_sequeezed_m,neg_labels_m)
girl_x,girl_y = combine_net_neg(pos_sequeezed_f,pos_labels_f,neg_sequeezed_f,neg_labels_f)

In [82]:
"""
train: boy 
test: girl 
"""
# train model with boy 
    # get x and y 
    # train 
train_x,train_y = get_x_y_train(boy_x,boy_y)
model.fit(train_x,train_y.ravel())
accs_g = []
# accuracys 
# loop girl shape[0]
    # get accuracy and append 
for i in range(girl_x.shape[0]):
    x_test,y_test = girl_x[i],girl_y[i]
    y_pred = model.predict(x_test)
    accs_g.append(accuracy_score(y_test,y_pred))

In [83]:
print(accs_g)

[0.7222222222222222, 0.8055555555555556, 0.6388888888888888, 0.6944444444444444, 0.6666666666666666, 0.6111111111111112, 0.5277777777777778, 0.7777777777777778, 0.7777777777777778, 0.6388888888888888, 0.7777777777777778, 0.6944444444444444, 0.7222222222222222, 0.5, 0.7777777777777778, 0.8333333333333334, 0.7, 0.7, 0.5333333333333333, 0.6388888888888888, 0.6666666666666666, 0.6944444444444444]


In [84]:
"""
train: boy 
test: girl 
"""
# train model with boy 
    # get x and y 
    # train 
train_x,train_y = get_x_y_train(girl_x,girl_y)
model.fit(train_x,train_y.ravel())
accs_b = []
# accuracys 
# loop girl shape[0]
    # get accuracy and append 
for i in range(boy_x.shape[0]):
    x_test,y_test = boy_x[i],boy_y[i]
    y_pred = model.predict(x_test)
    accs_b.append(accuracy_score(y_test,y_pred))

In [85]:
print(accs_b)

[0.7222222222222222, 0.6111111111111112, 0.7222222222222222, 0.6111111111111112, 0.7222222222222222, 0.7222222222222222, 0.8611111111111112, 0.6388888888888888, 0.6666666666666666, 0.6388888888888888, 0.8, 0.5, 0.5833333333333334, 0.6666666666666666, 0.6111111111111112, 0.6388888888888888, 0.6944444444444444, 0.5, 0.5833333333333334, 0.5833333333333334]


In [86]:
df = pd.read_csv('feb_25.csv')

In [87]:
df.columns

Index(['ID_boy', 'ID_girl', 'within_classification', 'stress', 'stressGroup',
       '42set_ID', 'between_classification_all', 'between_stressGroup',
       'between_gender', '39set_ID', 'between_classification',
       'between_stressGroup.1', '33set_ID', 'between_classification.1',
       'between_stressGroup.2', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'],
      dtype='object')

In [88]:
accs = accs_b 
for i in accs_g:
    accs.append(i)

In [89]:
print(len(accs))
# df['between_gender'] = accs

42


In [90]:
for i in range(len(accs)):
    accs[i] = round(accs[i],3)


In [92]:
for _ in range(22):
    accs.append(-1)

In [93]:
df["between_gender"] = accs

In [94]:
df.to_csv('feb_25.csv')

In [97]:
sum(accs[:42])/len(accs[:42])

0.6708571428571426

In [96]:
accs

[0.722,
 0.611,
 0.722,
 0.611,
 0.722,
 0.722,
 0.861,
 0.639,
 0.667,
 0.639,
 0.8,
 0.5,
 0.583,
 0.667,
 0.611,
 0.639,
 0.694,
 0.5,
 0.583,
 0.583,
 0.722,
 0.806,
 0.639,
 0.694,
 0.667,
 0.611,
 0.528,
 0.778,
 0.778,
 0.639,
 0.778,
 0.694,
 0.722,
 0.5,
 0.778,
 0.833,
 0.7,
 0.7,
 0.533,
 0.639,
 0.667,
 0.694,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]